In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files

# Upload your files (choose errors.txt and reference.txt when prompted)
uploaded = files.upload()


Saving errors.txt to errors.txt


In [ ]:

from google.colab import files

# Upload your files (choose errors.txt and reference.txt when prompted)
uploaded = files.upload()


Saving reference.txt to reference.txt


In [ ]:
!pip install rapidfuzz fuzzy


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.3 MB/s eta 0:00:00
  Created wheel for fuzzy: filename=Fuzzy-1.2.2-cp312-cp312-linux_x86_64.whl size=231379 sha256=0550695fc671d4caf057571d0224e3abf858aa340cce63b93870acc8d68a202b
  Stored in directory: /root/.cache/pip/wheels/ae/17/05/e688b455079b61cb255bc2e08458d6e66894a14be5cd5a5954
Successfully built fuzzy


In [ ]:

from google.colab import files

# Upload your files (choose errors.txt and reference.txt when prompted)
uploaded = files.upload()

Saving ground_truth_1.txt to ground_truth_1.txt


In [ ]:
from rapidfuzz import process, fuzz
import re

# Step A: Load files
with open("errors.txt", "r") as f:
    errors = [line.strip() for line in f]

with open("reference.txt", "r") as f:
    dictionary = [line.strip() for line in f]

# Step B: Normalization function
def normalize(word):
    word = word.lower()
    word = re.sub(r'(.)\1+', r'\1', word)  # remove repeated letters
    return word

# Step C: Correction function
def correct_word(word, dictionary):
    normalized = normalize(word)
    match, score, _ = process.extractOne(normalized, dictionary, scorer=fuzz.ratio)
    return match

# Step D: Create corrected list
output_lines = ["File_Error\tCorrected"]

for w in errors:
    corrected = correct_word(w, dictionary)
    output_lines.append(f"{w}\t{corrected}")

# Step E: Save output to file
with open("corrected.txt", "w") as f:
    f.write("\n".join(output_lines))


In [ ]:
# Step F: Load ground truth
with open("ground_truth_1.txt", "r") as f:
    ground_truth = [line.strip() for line in f]


In [ ]:
# Step G: Calculate accuracy
correct = 0
total = len(errors)

for i in range(total):
    predicted = correct_word(errors[i], dictionary).lower()
    actual = ground_truth[i].lower()

    if predicted == actual:
        correct += 1

accuracy = (correct / total) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 0.00%


In [ ]:
from google.colab import files
files.download("corrected.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install fuzzy


In [ ]:
def apply_custom_rules(word):
    rules = {
        "ph": "f",
        "bh": "b",
        "au": "aa"
    }
    for k, v in rules.items():
        word = word.replace(k, v)
    return word


In [ ]:
import fuzzy

# Use DMetaphone instead of Soundex or Metaphone
dmetaphone = fuzzy.DMetaphone()

def phonetic_code(word):
    # DMetaphone handles Unicode better
    return dmetaphone(word)

In [ ]:
from rapidfuzz import fuzz

def hybrid_score(word, candidate):
    # Apply rules + normalize
    w = apply_custom_rules(normalize(word))
    c = apply_custom_rules(normalize(candidate))

    # Fuzzy score
    fuzzy_score = fuzz.ratio(w, c)

    # Phonetic score (same = 100, else 0 for simplicity)
    phonetic_score = 100 if phonetic_code(w) == phonetic_code(c) else 0

    # Weighted hybrid score
    final_score = (0.7 * fuzzy_score) + (0.3 * phonetic_score)
    return final_score


def correct_word(word, dictionary):
    best_match = None
    best_score = -1
    for candidate in dictionary:
        score = hybrid_score(word, candidate)
        if score > best_score:
            best_match = candidate
            best_score = score
    return best_match


In [ ]:
dictionary = ["coffee", "bhajan", "aarti", "festival", "faizabad"]
errors = ["cofffe", "phestival", "bhaajan", "aartti"]

for w in errors:
    print(w, "→", correct_word(w, dictionary))


cofffe → coffee
phestival → festival
bhaajan → bhajan
aartti → aarti


In [ ]:
import fuzzy
print(dir(fuzzy))

['DMetaphone', 'Soundex', '__builtins__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__pyx_unpickle_DMetaphone', '__pyx_unpickle_Soundex', '__spec__', '__test__', '_non_AZ', '_nysiis_suffix_map', '_nysiis_trans_middle', '_nysiis_trans_not_first', '_nysiis_transforms', 'nysiis', 're']


In [ ]:
# Open input files
with open("errors.txt", "r") as f:
    errors = [line.strip() for line in f.readlines()]

with open("reference.txt", "r") as f:
    reference = [line.strip() for line in f.readlines()]

# Run correction
output_lines = ["File_Error\tCorrected\n"]
for word in errors:
    corrected = correct_word(word, reference)
    output_lines.append(f"{word}\t{corrected}\n")

# Save output
with open("output.txt", "w") as f:
    f.writelines(output_lines)

print("✅ Output file created: output.txt")


KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("output.txt")


FileNotFoundError: Cannot find file: output.txt

In [ ]:
from google.colab import files

# Upload your files (choose errors.txt and reference.txt when prompted)
uploaded = files.upload()


KeyboardInterrupt: 

In [ ]:
# Load ground truth (actual correct answers)
ground_truth_1 = {}
with open("ground_truth_1.txt", "r") as f:
    for line in f:
        error, actual = line.strip().split()
        ground_truth_1[error] = actual

# Compare program output with ground truth
correct = 0
total = 0
with open("output.txt", "r") as f:
    for line in f:
        error, predicted = line.strip().split()
        total += 1
        if ground_truth_1.get(error, "") == predicted:
            correct += 1

accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}%")
